In [14]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from DLC_for_WBFM.utils.projects.finished_project_data import ProjectData
import napari
import pandas as pd
import numpy as np
from torch.utils.data import random_split
from DLC_for_WBFM.utils.nn_utils.data_loading import NeuronTripletDataset
from torch.utils.data import DataLoader
import os
from tqdm.auto import tqdm
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
project_dir = "/scratch/zimmer/Charles/dlc_stacks/worm3-newseg-2021_11_17/"

In [16]:

training_folder = os.path.join(project_dir, 'nn_training')
training_dataset = NeuronTripletDataset(training_folder)
train_loader = DataLoader(training_dataset, batch_size=4)
small_dataset, large_dataset = random_split(training_dataset, [100, 900], generator=torch.Generator().manual_seed(42))

In [17]:
# PCA plot
test_loader = DataLoader(small_dataset, batch_size=1)
all_data = []
label_of_each_row = []
all_val_loss = []

triplets_to_visualize = 200

for step, (anchor_img, positive_img, pos_label, negative_img, neg_label) in enumerate(tqdm(test_loader, desc="Validating", leave=False)):
    anchor_out = model(anchor_img)
    positive_out = model(positive_img)
    negative_out = model(negative_img)
    all_data.append(anchor_out.detach().numpy().T)
    all_data.append(positive_out.detach().numpy().T)
    all_data.append(negative_out.detach().numpy().T)
    
    label_of_each_row.append(pos_label)
    label_of_each_row.append(pos_label)
    label_of_each_row.append(neg_label)
    
    loss = criterion(anchor_out, positive_out, negative_out)
    all_val_loss.append(loss.cpu().detach().numpy())
    
    if step > triplets_to_visualize:
        break

array_data = np.array(all_data)

Validating:   0%|          | 0/100 [00:00<?, ?it/s]

NameError: name 'model' is not defined